In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.impute import SimpleImputer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ufo-sightings/complete.csv
/kaggle/input/ufo-sightings/scrubbed.csv


In [2]:
ufos = pd.read_csv('../input/ufo-sightings/scrubbed.csv')
ufos.head()
#ufos.keys()
#ufos.dtypes

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [3]:
ufos['duration (seconds)'] = ufos['duration (seconds)'].str.strip('`')
ufos['duration (seconds)'] = ufos['duration (seconds)'].astype(float)

In [4]:
# Brute Force typecasting
ufos['latitude'][ufos['latitude'] == '33q.200088'] = '33.200088'
ufos['latitude'] = ufos['latitude'].astype(float)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
# Replace missing data with mode
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
ufos_temp = imputer.fit_transform(ufos)
ufos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              80332 non-null  object 
 1   city                  80332 non-null  object 
 2   state                 74535 non-null  object 
 3   country               70662 non-null  object 
 4   shape                 78400 non-null  object 
 5   duration (seconds)    65536 non-null  float64
 6   duration (hours/min)  80332 non-null  object 
 7   comments              80317 non-null  object 
 8   date posted           80332 non-null  object 
 9   latitude              80332 non-null  float64
 10  longitude             80332 non-null  float64
dtypes: float64(3), object(8)
memory usage: 6.7+ MB


In [6]:
titles = ['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude ']
uf = pd.DataFrame(ufos_temp, columns=titles)
uf.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1949 21:00,lackland afb,tx,us,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),ca,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611


In [7]:
# Creating a new dataframe with specific features.
ufos_true = pd.DataFrame({'Seconds': uf['duration (seconds)'], 'Country': uf['country'],'Latitude': uf['latitude'],'Longitude': uf['longitude '], 'Shape': uf['shape']})


In [8]:
ufos_true.Country.unique()

array(['us', 'gb', 'ca', 'au', 'de'], dtype=object)

In [9]:
# Being pedantic, but doesn't hurt to check.
ufos_true.dropna(inplace=True)

In [10]:
ufos_true

,Seconds,Country,Latitude,Longitude,Shape
0,2700.0,us,29.883056,-97.941111,cylinder
1,7200.0,us,29.38421,-98.581082,light
2,20.0,gb,53.2,-2.916667,circle
3,20.0,us,28.978333,-96.645833,circle
4,900.0,us,21.418056,-157.803611,light
...,...,...,...,...,...
80327,300.0,us,36.165833,-86.784444,light
80328,300.0,us,43.613611,-116.2025,circle
80329,300.0,us,38.297222,-122.284444,other
80330,300.0,us,38.901111,-77.265556,circle


In [11]:
# Encoding Categorical features
from sklearn.preprocessing import LabelEncoder
ufos_true['Country'] = LabelEncoder().fit_transform(ufos_true['Country'])
ufos_true['Shape'] = LabelEncoder().fit_transform(ufos_true['Shape'])

In [12]:
# Dropping Shape due to mislabelling.
ufos_true[ufos_true['Shape']!= 0]
#ufos_true.drop(['Shape'], axis=1,inplace=True)

,Seconds,Country,Latitude,Longitude,Shape
0,2700.0,4,29.883056,-97.941111,8
1,7200.0,4,29.38421,-98.581082,19
2,20.0,3,53.2,-2.916667,4
3,20.0,4,28.978333,-96.645833,4
4,900.0,4,21.418056,-157.803611,19
...,...,...,...,...,...
80327,300.0,4,36.165833,-86.784444,19
80328,300.0,4,43.613611,-116.2025,4
80329,300.0,4,38.297222,-122.284444,20
80330,300.0,4,38.901111,-77.265556,4


In [13]:
ufos_true.describe


<bound method NDFrame.describe of       Seconds  Country   Latitude   Longitude  Shape
0      2700.0        4  29.883056  -97.941111      8
1      7200.0        4   29.38421  -98.581082     19
2        20.0        3       53.2   -2.916667      4
3        20.0        4  28.978333  -96.645833      4
4       900.0        4  21.418056 -157.803611     19
...       ...      ...        ...         ...    ...
80327   300.0        4  36.165833  -86.784444     19
80328   300.0        4  43.613611   -116.2025      4
80329   300.0        4  38.297222 -122.284444     20
80330   300.0        4  38.901111  -77.265556      4
80331   300.0        4  35.652778  -97.477778      3

[80332 rows x 5 columns]>

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# getting features and splitting them.
features = ['Seconds', 'Latitude', 'Longitude']

X = ufos_true[features]
y = ufos_true['Country']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [15]:
# instantiating and fitting, and predicting with the model
pipe = Pipeline([('scaler', StandardScaler()),('model', LogisticRegression(max_iter=500))])
#model = LogisticRegression()
pipe.fit(X_train, y_train)
predictions = pipe.predict(X_test)

In [16]:
print(classification_report(y_test, predictions))
print('Predicted labels: ', predictions)
print('Accuracy: ', accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       0.75      0.93      0.83       175
           1       0.10      0.01      0.02      1048
           2       0.00      0.00      0.00        33
           3       0.66      0.96      0.78       656
           4       0.96      0.98      0.97     24598

    accuracy                           0.94     26510
   macro avg       0.49      0.58      0.52     26510
weighted avg       0.91      0.94      0.92     26510

Predicted labels:  [4 3 4 ... 4 4 4]
Accuracy:  0.9401735194266314


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
import pickle
model_filename = 'ufo-model.pkl'
pickle.dump(pipe, open(model_filename,'wb'))


model = pickle.load(open('ufo-model.pkl','rb'))
print(pipe.predict([[50,44,-12]]))

[4]


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
